In [1]:
import matplotlib.pyplot as plt
import numpy as np
import time
import csv
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.recurrent import LSTM

import pandas as pd
from random import random

%matplotlib inline
import matplotlib.pyplot as plt
np.random.seed(1234)



Using Theano backend.


In [2]:
data = pd.read_csv("../data/processed/train.csv")
#data.describe()
list(data.columns.values[7550:])

['V993',
 'V994',
 'V995',
 'V996',
 'V997',
 'V998',
 'V999',
 'V1000',
 'Activity',
 'Social']

In [3]:
list(data.columns.values[0:10])

['Unnamed: 0',
 'session',
 'timestamp',
 'value.Mean',
 'value.SD',
 'value.Fix',
 'value.Sac',
 'value.Fix.Dur',
 'value.Fix.Disp',
 'value.Sac.Dur']

In [4]:
# Add dummy variables for the target variables
#dumAct = pd.get_dummies(data.Activity)
#dumSoc = pd.get_dummies(data.Social)
#datadum = pd.concat([data, dumAct, dumSoc], axis=1)
#plt.figure()
#datadum['value.Mean'][1:100].plot()
#datadum['EXP'][1:100].plot()
#datadum['MON'][1:100].plot()
testing = pd.read_csv("../data/processed/test.csv")
#testing.describe()

In [10]:
#print data['Activity']
print data.ix[:,0:10]

    Unnamed: 0                       session  timestamp  value.Mean  value.SD  \
0          462  case1-day1-session2-teacher1       5000    4.314267  0.306538   
1          463  case1-day1-session2-teacher1      10000    4.345133  0.287732   
2          464  case1-day1-session2-teacher1      15000    4.443455  0.452320   
3          465  case1-day1-session2-teacher1      20000    4.220532  0.548338   
4          466  case1-day1-session2-teacher1      25000    3.732533  0.318437   
5          467  case1-day1-session2-teacher1      30000    3.518267  0.292483   
6          468  case1-day1-session2-teacher1      35000    3.387708  0.313086   
7          469  case1-day1-session2-teacher1      40000    3.529435  0.568454   
8          470  case1-day1-session2-teacher1      45000    3.851000  0.527294   
9          471  case1-day1-session2-teacher1      50000    3.763123  0.483854   
10         472  case1-day1-session2-teacher1      55000    3.398472  0.505439   
11         473  case1-day1-s

In [ ]:
#TODO: split into train and test sets
#Follow/mix:
#https://github.com/Vict0rSch/deep_learning/tree/master/keras/recurrent
#https://gist.github.com/hnykda/c362f0ad488e3b289394
#https://github.com/fchollet/keras/blob/master/examples/imdb_lstm.py

# Prepare the data
# We cut out the last samples to match a multiple of our batch size (20)
x_train = data.ix[0:4499,3:(data.shape[1]-2)]
y_train = pd.get_dummies(data.Activity[0:4500]) 
print x_train.shape
print y_train.shape
x_val = testing.ix[0:1039,3:(testing.shape[1]-2)]
y_val = pd.get_dummies(testing.Activity[0:1040])
print x_val.shape
print y_val.shape
# From https://keras.io/getting-started/sequential-model-guide/\
data_dim = 7555
timesteps = 10
nb_classes = y_train.shape[1]
batch_size = 20

# expected input batch shape: (batch_size, timesteps, data_dim)
# note that we have to provide the full batch_input_shape since the network is stateful.
# the sample of index i in batch k is the follow-up for the sample i in batch k-1.

model = Sequential()
model.add(LSTM(200, return_sequences=True, stateful=True,
               batch_input_shape=(batch_size, timesteps, data_dim)))
model.add(LSTM(100, return_sequences=True, stateful=True))
model.add(LSTM(50, stateful=True))
model.add(Dense(nb_classes, activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])
model.fit(x_train, y_train,
          batch_size=batch_size, nb_epoch=1,
          validation_data=(x_val, y_val))
score = model.evaluate(x_val, y_val, batch_size=batch_size)
print scor